In [7]:
from readers.populationreader import PopulationReader
from caches.lfucache import LFUCache
from cachesim.tools import PBarMixIn
from cachesim import Status

In [11]:
count = 1000000
cbase = count // 10


def hns(n: int, s: float) -> float:
    return sum([k ** -s for k in range(1, n + 1)])


print("Create population...")
s = 1.3
HNs = hns(cbase, s)
reader = PopulationReader(count, cbase, weights=[k ** -s / HNs for k in range(1, cbase + 1)])

Create population...


In [9]:
sizer = 0.1


class MyCache(PBarMixIn, LFUCache):
    pass


cache = MyCache(totalsize=cbase * sizer)

req, age = zip(*list(cache.map(reader)))

MyCache: 100%|██████████| 1000000/1000000 [00:02<00:00, 402992.15it/s]


In [16]:
hit = sum(map(lambda r: r.status == Status.HIT, req))
print(f"Requests: {len(req)}")
print(f"CHR: {hit / len(req) * 100:.2f}% (expected: {hns(int(cbase * sizer), s) / hns(cbase, s) * 100:.2f}%)")
print(f"Bytes sent: {sum(r.size for r in req)} B")

x = [r.time for r in req]

Requests: 1000000
CHR: 95.63% (expected: 97.26%)
Bytes sent: 1000000 B
